In [2]:
# Copyright 2019 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [3]:
from sagemaker.tensorflow import TensorFlow

In [4]:
import sagemaker
bucket = sagemaker.Session().default_bucket() # Any S3 bucket can be specified but we're using the Sagemaker default bucket here.
prefix = 'sagemaker/ml-model-migration'


# Remote Execution

In [ ]:
role = sagemaker.get_execution_role() 
tf_estimator = TensorFlow(py_version='py3', 
                          framework_version='1.12', 
                          entry_point='task.py',
                          role=role,
                          train_instance_count=1,
                          train_instance_type='ml.m4.xlarge',
                          source_dir='training_job/')

In [6]:
tf_estimator.fit({'train':'s3://sagemaker-us-east-2-708267171719/sagemaker/ml-model-migration/data/mnist/mnist_train.tfrecords',
                  'eval':'s3://sagemaker-us-east-2-708267171719/sagemaker/ml-model-migration/data/mnist/mnist_test.tfrecords'})

Creating tmptypeq2q0_algo-1-vgrzu_1 ... 
Attaching to tmptypeq2q0_algo-1-vgrzu_12mdone
algo-1-vgrzu_1  | 2019-07-11 21:27:55,502 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
algo-1-vgrzu_1  | 2019-07-11 21:27:55,518 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-vgrzu_1  | 2019-07-11 21:27:56,030 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-vgrzu_1  | 2019-07-11 21:27:56,061 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-vgrzu_1  | 2019-07-11 21:27:56,075 sagemaker-containers INFO     Invoking user script
algo-1-vgrzu_1  | 
algo-1-vgrzu_1  | Training Env:
algo-1-vgrzu_1  | 
algo-1-vgrzu_1  | {
algo-1-vgrzu_1  |     "additional_framework_parameters": {},
algo-1-vgrzu_1  |     "channel_input_dirs": {
algo-1-vgrzu_1  |         "train": "/opt/ml/input/data/train",
algo-1-vgrzu_1  |         "eval": "/opt/ml/input/data/eval"
algo-1-vgrzu

algo-1-vgrzu_1  | INFO:tensorflow:Calling model_fn.
algo-1-vgrzu_1  | INFO:tensorflow:Done calling model_fn.
algo-1-vgrzu_1  | INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='s3://sagemaker-us-east-2-708267171719/sagemaker-tensorflow-scriptmode-2019-07-11-21-27-46-165/model/keras/keras_model.ckpt', vars_to_warm_start='.*', var_name_to_vocab_info={}, var_name_to_prev_var_name={})
algo-1-vgrzu_1  | INFO:tensorflow:Warm-starting from: ('s3://sagemaker-us-east-2-708267171719/sagemaker-tensorflow-scriptmode-2019-07-11-21-27-46-165/model/keras/keras_model.ckpt',)
algo-1-vgrzu_1  | INFO:tensorflow:Warm-starting variable: conv2d/kernel; prev_var_name: Unchanged
algo-1-vgrzu_1  | INFO:tensorflow:Warm-starting variable: conv2d/bias; prev_var_name: Unchanged
algo-1-vgrzu_1  | INFO:tensorflow:Warm-starting variable: conv2d_1/kernel; prev_var_name: Unchanged
algo-1-vgrzu_1  | INFO:tensorflow:Warm-starting variable: conv2d_1/bias; prev_var_name: Uncha

tmptypeq2q0_algo-1-vgrzu_1 exited with code 0
Aborting on container exit...
===== Job Complete =====
